# Train

TODO convert this to train.py

In [1]:
import os
from pathlib import Path
import math
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from data_loader.cityscapes import CityscapesDataLoader
from model.deeplabv3plus import DeepLabv3Plus
from torch_poly_lr_decay import PolynomialLRDecay
from model.metric import SegmentationMetrics

In [2]:
def save_checkpoint(epoch, model, optimizer, path):
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        path,
    )


def train_one_epoch(epoch, model, criterion, optimizer, scheduler, data_loader, device):
    model.train()

    total_loss = 0
    n_batches = len(data_loader)
    for i, sample in enumerate(tqdm(data_loader)):
        images = sample["image"].to(device)
        masks = sample["mask"].to(device)

        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, masks)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        writer.add_scalar("train_iter_loss", loss.item(), i + n_batches * epoch)

    avg_loss = total_loss / n_batches
    writer.add_scalar("train_epoch_avg_loss", avg_loss, epoch)
    return avg_loss


def validate(epoch, model, criterion, data_loader, num_classes, device):
    model.eval()

    total_loss = 0
    n_batches = len(data_loader)
    metrics = SegmentationMetrics(num_classes=num_classes, ignore_idx=ignore_idx)
    with torch.no_grad():
        for i, sample in enumerate(tqdm(data_loader)):
            images = sample["image"].to(device)
            masks = sample["mask"].to(device)

            pred = model(images)
            loss = criterion(pred, masks)
            total_loss += loss.item()

            pred_cls = torch.argmax(pred, dim=1)
            metrics.update(pred_cls, masks)

    model.train()

    avg_loss = total_loss / n_batches
    ious, mIoU = metrics.iou()
    writer.add_scalar("val_epoch_avg_loss", avg_loss, epoch)
    writer.add_scalar("val_epoch_mIoU", mIoU, epoch)
    return avg_loss, ious, mIoU

## Checkpoints and Tensorboard

In [3]:
run_dir = "run3"

log_dir = os.path.join("./experiments", "runs", run_dir)
Path(log_dir).mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)
print(f"Writing tensorboard logs to {log_dir}")

checkpoint_dir = os.path.join("./experiments", "checkpoints", run_dir)
Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
checkpoint_file = os.path.join(checkpoint_dir, "best_model.pt")
print(f"Saving model checkpoints to {checkpoint_file}")

Writing tensorboard logs to ./experiments/runs/run3
Saving model checkpoints to ./experiments/checkpoints/run3/best_model.pt


## Get DataLoader

In [4]:
img_root = "./data/leftImg8bit"
mask_root = "./data/gtFine"
batch_size = 4

cityscapes = CityscapesDataLoader(img_root, mask_root, batch_size)

## Initialize Model, Loss, and Optimizer

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = cityscapes.train_set.num_classes
ignore_idx = cityscapes.train_set.ignoreId
initial_lr = 0.007
end_lr = 0.001
momentum = 0.9
weight_decay = 0.0005
train_iterations = 90000
total_epochs = math.ceil(train_iterations / len(cityscapes.train_loader))

model = DeepLabv3Plus(num_classes).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=ignore_idx)
optimizer = torch.optim.SGD(
    model.parameters(), lr=initial_lr, momentum=momentum, weight_decay=weight_decay
)
scheduler = PolynomialLRDecay(
    optimizer, max_decay_steps=train_iterations, end_learning_rate=end_lr, power=0.9
)

In [6]:
# patience = 15
best_mIoU = 0
best_epoch = 0
for epoch in range(total_epochs):
    train_loss = train_one_epoch(
        epoch, model, criterion, optimizer, scheduler, cityscapes.train_loader, device
    )
    val_loss, ious, mIoU = validate(
        epoch, model, criterion, cityscapes.val_loader, num_classes, device
    )
    print(f"Epoch {epoch}: val_loss {val_loss:.4f} | mIoU {mIoU: .4f}")

    if mIoU > best_mIoU:
        best_mIoU = mIoU
        best_epoch = epoch
        save_checkpoint(epoch, model, optimizer, checkpoint_file)
    # elif epoch - best_epoch >= patience:
    #     print(f"Early Stopping at epoch {epoch}")
    #     break

100%|██████████| 125/125 [01:43<00:00,  1.21it/s]


Epoch 0: val_loss 0.8569 | mIoU  0.1491


100%|██████████| 125/125 [01:43<00:00,  1.21it/s]


Epoch 1: val_loss 0.7237 | mIoU  0.1772


100%|██████████| 125/125 [01:42<00:00,  1.22it/s]


...

Epoch 118: val_loss 0.1776 | mIoU  0.5254


100%|██████████| 125/125 [01:44<00:00,  1.20it/s]


Epoch 119: val_loss 0.1738 | mIoU  0.5382


100%|██████████| 125/125 [01:43<00:00,  1.20it/s]

Epoch 120: val_loss 0.1708 | mIoU  0.5345


In [7]:
writer.close()

In [12]:
writer = SummaryWriter(log_dir=log_dir)

In [13]:
print(total_epochs)
print(epoch)
prev_epochs = 120
for epoch in range(prev_epochs+1, prev_epochs+81):
    train_loss = train_one_epoch(
        epoch, model, criterion, optimizer, scheduler, cityscapes.train_loader, device
    )
    val_loss, ious, mIoU = validate(
        epoch, model, criterion, cityscapes.val_loader, num_classes, device
    )
    print(f"Epoch {epoch}: val_loss {val_loss:.4f} | mIoU {mIoU: .4f}")

    if mIoU > best_mIoU:
        best_mIoU = mIoU
        best_epoch = epoch
        save_checkpoint(epoch, model, optimizer, checkpoint_file)

  0%|          | 0/744 [00:00<?, ?it/s]

121
120


  0%|          | 0/744 [00:00<?, ?it/s]

Epoch 121: val_loss 0.1759 | mIoU  0.5225


100%|██████████| 125/125 [01:43<00:00,  1.21it/s]


...

Epoch 159: val_loss 0.1708 | mIoU  0.5321


  0%|          | 0/744 [00:00<?, ?it/s]

Epoch 160: val_loss 0.1666 | mIoU  0.5481


  0%|          | 2/744 [00:01<08:46,  1.41it/s]


KeyboardInterrupt: 